## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: string (nullable = true)

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select * from facilities where membercost > 0

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
4,Massage Room 1,9.9,80.0,4000,3000
5,Massage Room 2,9.9,80.0,4000,3000
6,Squash Court,3.5,17.5,5000,80


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
select count(name) from (
select * from facilities where membercost = 0)


count(name)
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
select * from
(
select *, 
membercost / monthlymaintenance as percent_fee
from facilities 
where membercost >0 
)
where percent_fee < .2

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,percent_fee
0,Tennis Court 1,5.0,25.0,10000,200,0.025
1,Tennis Court 2,5.0,25.0,8000,200,0.025
4,Massage Room 1,9.9,80.0,4000,3000,0.0033
5,Massage Room 2,9.9,80.0,4000,3000,0.0033
6,Squash Court,3.5,17.5,5000,80,0.04375


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
select * from facilities where facid in (0,5)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
select name, 
monthlymaintenance,
case when monthlymaintenance > 100 then "expensive" else "cheap" end as label
from facilities

name,monthlymaintenance,label
Tennis Court 1,200,expensive
Tennis Court 2,200,expensive
Badminton Court,50,cheap
Table Tennis,10,cheap
Massage Room 1,3000,expensive
Massage Room 2,3000,expensive
Squash Court,80,cheap
Snooker Table,15,cheap
Pool Table,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
select 
firstname,
surname
from members 
order by joindate desc

firstname,surname
Darren,Smith
Erica,Crumpet
John,Hunt
Hyacinth,Tupperware
Millicent,Purview
Henry,Worthington-Smyth
David,Farrell
Henrietta,Rumney
Douglas,Jones
Ramnaresh,Sarwin


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
select * from facilities where name like '%Tennis%'

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
3,Table Tennis,0.0,5.0,320,10


In [0]:
%sql
select * from bookings

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2
3,7,1,2012-07-03 19:00:00,2
4,8,1,2012-07-03 10:00:00,1
5,8,1,2012-07-03 15:00:00,1
6,0,2,2012-07-04 09:00:00,3
7,0,2,2012-07-04 15:00:00,3
8,4,3,2012-07-04 13:30:00,2
9,4,0,2012-07-04 15:00:00,2


In [0]:
%sql
select * from members

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,GUEST,GUEST,GUEST,0,(000) 000-0000,null,2012-07-01 00:00:00
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,null,2012-07-02 12:02:05
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,null,2012-07-02 12:08:23
3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,null,2012-07-03 09:32:15
4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05
5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1,2012-07-09 10:44:09
6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,null,2012-07-15 08:52:55
7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4,2012-07-25 08:59:12
8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3,2012-07-25 16:02:35
9,Stibbons,Ponder,"5 Dragons Way, Winchester",87630,(833) 160-3900,6,2012-07-25 17:09:05


In [0]:
%sql
select 
full_information 

from (
select
-- bookings.memid,
-- bookings.facid,
-- facilities.name,
concat(facilities.name, " ", members.firstname, " ", members.surname) as full_information
-- members.surname,
-- members.firstname
from bookings
left join members on bookings.memid = members.memid
left join facilities on bookings.facid = facilities.facid
where bookings.facid in (0,1)
)
group by 1

full_information
Tennis Court 2 GUEST GUEST
Tennis Court 1 Burton Tracy
Tennis Court 2 Ponder Stibbons
Tennis Court 1 David Jones
Tennis Court 1 Anne Baker
Tennis Court 1 Matthew Genting
Tennis Court 2 Ramnaresh Sarwin
Tennis Court 1 John Hunt
Tennis Court 1 Joan Coplin
Tennis Court 1 Timothy Baker


In [0]:
%%md
#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

UsageError: Cell magic `%%md` not found.

In [0]:
%sql
-- don't know how to do this without a subquery

#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
select
* from (
select 
bookings.bookid,
bookings.memid,
bookings.slots,
facilities.membercost,
facilities.guestcost,
bookings.starttime,
case when bookings.memid = 0 then (bookings.slots * facilities.guestcost) else (bookings.slots * facilities.membercost) end as total_cost
from bookings
left join facilities on bookings.facid = facilities.facid
where starttime like '%2012-09-14%'
)
where total_cost > 30
order by total_cost desc


bookid,memid,slots,membercost,guestcost,starttime,total_cost
2946,0,4,9.9,80.0,2012-09-14 11:00:00,320.0
2937,0,2,9.9,80.0,2012-09-14 09:00:00,160.0
2942,0,2,9.9,80.0,2012-09-14 16:00:00,160.0
2940,0,2,9.9,80.0,2012-09-14 13:00:00,160.0
2926,0,6,5.0,25.0,2012-09-14 17:00:00,150.0
2925,0,3,5.0,25.0,2012-09-14 14:00:00,75.0
2922,0,3,5.0,25.0,2012-09-14 19:00:00,75.0
2920,0,3,5.0,25.0,2012-09-14 16:00:00,75.0
2948,0,4,3.5,17.5,2012-09-14 09:30:00,70.0
2941,13,4,9.9,80.0,2012-09-14 14:00:00,39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
select
name,
total_cost_sum
from (
select
name,
sum(total_cost) as total_cost_sum
from (
select 
bookings.bookid,
bookings.memid,
bookings.slots,
facilities.membercost,
facilities.guestcost,
facilities.name,
bookings.starttime,
case when bookings.memid = 0 then (bookings.slots * facilities.guestcost) else (bookings.slots * facilities.membercost) end as total_cost
from bookings
left join facilities on bookings.facid = facilities.facid
)
group by 1
)
where total_cost_sum < 1000
order by 2

name,total_cost_sum
Table Tennis,180.0
Snooker Table,240.0
Pool Table,270.0
